# 1. Importing data and libraries <a class="anchor" id="1"></a>

In [22]:
import pandas as pd
import datetime

In [23]:
dtype_dict = {
    'ProductFamily_ID':'category',
    'ProductCategory_ID':'category',
    'ProductBrand_ID':'category',
    'ProductName_ID':'category',
    'ProductPackSKU_ID':'category',
    'Point-of-Sale_ID':'category',
    'Measures':'category',
    'Quantity':'int8',
    'Total_Sales':'float32'
}
path = r'D:\NOVAIMS_MAA\NOVAIMS_MAA_final.csv'

In [24]:
df = pd.read_csv(path,usecols=["ProductName_ID", "Point-of-Sale_ID","Total_Sales"],dtype=dtype_dict)
# data = pd.read_csv('dataset/NOVAIMS_MAA_2020e21_BusinessCasesDataScience_MindOverData_RetailChallenge.csv', iterator=True, chunksize=1000)
df

,ProductName_ID,Point-of-Sale_ID,Total_Sales
0,649,1,1540.0
1,649,1,3080.0
2,649,1,1540.0
3,649,1,1620.0
4,649,1,1620.0
...,...,...,...
91187501,577,41,643.0
91187502,577,41,638.0
91187503,577,41,652.0
91187504,577,41,643.0


We have the same number of 'Sell-out values' and 'Sell-out units', meaning that it's always record byt pair of records.

\
**Idea**\
Creating for each combination of ProductFamily_ID, ProductCategory_ID, ProductBrand_ID, ProductName_ID, ProductPackSKU_ID, Point-of-Sale_ID and Date one column for the quantity and one column for the price in a single record

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91187506 entries, 0 to 91187505
Data columns (total 3 columns):
 #   Column            Dtype   
---  ------            -----   
 0   ProductName_ID    category
 1   Point-of-Sale_ID  category
 2   Total_Sales       float32 
dtypes: category(2), float32(1)
memory usage: 695.8 MB


In [10]:
df.describe(include='all')

,ProductName_ID,Point-of-Sale_ID,Total_Sales
count,91187506,91187506,9.118751e+07
unique,2127,333,NaN
top,269,13,NaN
freq,907648,1030029,NaN
mean,NaN,NaN,3.517875e+03
std,NaN,NaN,6.656592e+03
min,NaN,NaN,-1.000000e+01
25%,NaN,NaN,8.600000e+02
50%,NaN,NaN,1.654000e+03
75%,NaN,NaN,3.416000e+03


In [27]:
df['Point-of-Sale_ID'].unique()

['1', '2', '3', '4', '5', ..., '395', '396', '397', '398', '399']
Length: 333
Categories (333, object): ['1', '2', '3', '4', ..., '396', '397', '398', '399']

In [26]:
df[df['Point-of-Sale_ID']=='1'].groupby('ProductName_ID').sum().sort_values(by='Total_Sales',ascending = False)[:10]

,Total_Sales
ProductName_ID,
282,190542000.0
738,135320288.0
1277,104524888.0
847,102554024.0
567,80696328.0
159,73247600.0
1956,67090444.0
1422,59840416.0
2592,46912296.0
